#Lake Windermere eNDA metabarcoding data 

In this exercise we will use [metaBEAT](https://github.com/HullUni-bioinformatics/metaBEAT), a tool tailored towards reproducible and efficient analyses of metabarcoding data that we have developed in-house. It is still under active development and will likely be extended further in the future. The pipeline is available in a Docker [container](https://registry.hub.docker.com/u/chrishah/metabeat/) with all necessary dependencies. The Docker image is building on [ReproPhylo](https://registry.hub.docker.com/u/szitenberg/reprophylo/).

The data we will be analyzing are CytB sequences amplified from eDNA samples collected from Lake Windermere. The experiment was designed to assess the potential of the eDNA approach to assess fish community compositions and was published recently (Haenfling et al. 2016). 
The metaBEAT tool is designed as a wrapper around a complete analysis from raw data, performing (optionally) de-multiplexing, quality filtering, clustering along the way, to OTU tables in biom format. It currently supports BLAST and phylogenetic placement (pplacer). The plan is to include further approaches in the future and to allow for efficient and standardized comparative assessments of all approaches. 

metBEAT offers a large number of options. Most of them will sound familiar and should make sense to you given your experience from the course so far. 


In [ ]:
!metaBEAT_global.py -h

We will limit ourselves to a basic processing (read trimming, -merging-, clustering) for now.

Minimum input for an anlysis is a set of query sequences in one or several files (accepted are a number of file formats, e.g. `fasta`, `fastq`). These will be run through the pipeline sequentially.

You will need to provide information on the nature and location of the query sequence files in a separate tab-delimited text files via the `-Q` flags,

The format for this text file is as follows:

`unique sample_ID <tab> format <tab> file1 <tab> file2 (optional) <tab> optionally barcodes`

You may generate the required text files in any text editor. But for the sake of reproducibility, let's use a mini python script to programatically produce this file, which I will call `Querymap.txt`.

In [ ]:
import os

out_list = []
string = ''
datadir = '../raw_Illumina_data/reads/'
files = os.listdir(datadir)
for f in sorted(files):
    if '_1.fastq' in f:
        string += "%s\tfastq\t%s%s\t%s/%s" %(f.replace('-CytB_1.fastq.gz', ''), datadir, f, datadir, f.replace('_1.fastq', '_2.fastq'))+'\n'

out = open('Querymap.txt', 'w')
out.write(string)
out.close()

Have a look (note that the output is probably line-wrapped). Another example can be found [here](https://github.com/HullUni-bioinformatics/metabarcode-course-2016/blob/master/data/exercise-3/example_data/Querymap.txt).

In [ ]:
!head Querymap.txt

In [ ]:
%%bash

echo -e "Starttime: $(date)\n"

metaBEAT_global.py \
-Q Querymap.txt \
--trim_qual 30 \
--trim_minlength 100 \
--merge \
--forward_only \
--product_length 400 \
--cluster --clust_match 1 --clust_cov 3 \
-m CytB -o CytB-trim30min100-merge-c3-id1 \
-n 5 -v &> log

echo -e "Endtime: $(date)\n"

Read processing will take about 20-30 minutes.

Some more details about the above analysis:

```bash
metaBEAT_global.py \ #call the program
-Q Querymap.txt \ #path to the Querymap
--trim_qual 30 \ #minimum quality for trimming is prhed Q30 (trimmomatic)
--trim_minlength 100 \ #only retain reads longer than 100 bp after trimming
--merge \ #merge overlapping read pairs (flash)
--forward_only \ #retain only forward read in cases where read pairs don't merge
--product_length 400 \ #expected length of merged read is roughly 400 bp
--cluster \ #cluster sequences (vsearch)
--clust_match 1 \ #cluster at 100 % identity
--clust_cov 3 \ #minimum coverage of 3 reads to retain cluster
-m CytB \ #arbitrary name of the marker
-o CytB-trim30min100-merge-c3-id1 \ #prefix for results files
-n 5 \ #use 5 threads
-v &> log #write verbose output to file called log
```

While the read processing is running we can continue with the course program. Next we will focus on constructing/curating a custom reference database.



 metaBEAT will generate a directory with all temporary files that were create during the processing for each sample and will record useful stats summarizing the data processing in the file `CytB-trim30min100-merge-c3-id1_read_stats.csv`.
 
You can explore the table or we can quickly plot out some of these stats here in the notebook, e.g. like so:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

# parse metaBEAT_read_stats.csv
stats_file = 'CytB-trim30min100-merge-c3-id1_read_stats.csv'

################
## prepare data
total_pairs = []
trimmed_pairs = []
trimmed_orphams = []
merged_pairs = []
queries = []

for l in open(stats_file, 'r').readlines()[1:]:
    smpl, tot, trimtot, trimpe, trimorph, merged, n1,n2,n3,n4, q = l.rstrip().split(',')
    total_pairs.append(int(tot)/2.0)
    trimmed_pairs.append(int(trimpe)/2.0)
    trimmed_orphams.append(int(trimorph))
    merged_pairs.append(int(merged))
    queries.append(int(q))
    
dat = [total_pairs, trimmed_pairs, trimmed_orphams, merged_pairs, queries]
dat = [np.array(d) for d in dat]
x = ['raw pairs', 'trimmed pairs', 'trimmed orphans', 'merged pairs', 'queries']

################
## do the plotting
fig = plt.figure(1, figsize=(12, 6))
ax = fig.add_subplot(111)
bp = ax.boxplot(dat)
bp = ax.set_xticklabels(x, rotation=45)


Detailed info on the entire process you can find in the log file `log`. It contains the exact commands that were run for each sample during each step of the process.

It's a large text file - just have a look at the first 100 lines for a start. 

In [ ]:
!head -n 100 log

__WELL DONE!__